# Chapter 5 Numerically Stable $QR$-Decomposition

In [ ]:
# numerical and scientific computing libraries
import numpy as np
import scipy as sp

# plotting libraries 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
# for pretty printing
np.set_printoptions(4, linewidth=100, suppress=True)

Recall the classical $QR$-decomposition implemented Chapter 4.

In [18]:
def classical_QR(A):
    m, n = A.shape

    R = np.zeros((n,n))
    R[0,0] = np.linalg.norm(A[:,0])

    Q = (1/R[0,0])*A[:,0]

    # Gram-Schmidt process
    for j in range(1,n):
        Q = np.column_stack((Q,A[:,j]))
        for i in range(j):
            R[i,j] = np.dot(Q[:,i].T,A[:,j])
            Q[:,j] = Q[:,j] - R[i,j]*Q[:,i]
        R[j,j] = np.linalg.norm(Q[:,j])
        if np.abs(R[j,j]) < 1e-10:
            raise ValueError("QR factorization failed: A is rank deficient")

        Q[:,j] = (1/R[j,j])*Q[:,j]

    return Q, R

An implementation of the modified $QR$-decomposition multiplying matrices of rank $m-1$.

In [19]:
def modified_QR(A):
    m, n = A.shape
    Q = np.copy(A).astype(float)  # Initialize Q with a copy of A
    R = np.zeros((n, n))

    for j in range(n):
        v_j = Q[:, j]
        for i in range(j):
            R[i, j] = np.dot(Q[:, i].T, v_j)
            v_j = v_j - R[i, j] * Q[:, i]
        R[j, j] = np.linalg.norm(v_j)
        if np.abs(R[j, j]) < 1e-10:
            raise ValueError("QR factorization failed: A is rank deficient")
        Q[:, j] = v_j / R[j, j]

    return Q, R

Let us try this modified decomposition to randomly generated instances.

In [20]:
# Setting dimension m >= n 
m = 1000
n = 990

# Generating a random matrix A  of dimension m x n
A = np.random.randn(m,n)

Q, R = modified_QR(A)

# Checking the QR factorization
print(np.allclose(A,Q@R))
print(np.allclose(Q.T@Q,np.eye(n)))
print(R[:min(10,n),:min(10,n)]) 

True
True
[[32.0925 -0.2228  0.2194  0.4978 -0.0672 -0.8269 -0.7415 -1.0651  0.9199  0.655 ]
 [ 0.     32.4641 -0.8934  1.0859 -0.8635 -0.3909 -1.2585  0.9591  0.0474 -0.0617]
 [ 0.      0.     31.6821 -0.0489  0.403  -0.9567  0.5652 -0.5471  0.5985  0.5688]
 [ 0.      0.      0.     31.5233  1.4921  1.2506  0.5738 -1.0222 -1.9273 -0.9437]
 [ 0.      0.      0.      0.     32.0945 -0.362  -0.1296  0.4852  0.2898 -0.3077]
 [ 0.      0.      0.      0.      0.     31.4496 -1.47    0.0563  0.2487  0.7918]
 [ 0.      0.      0.      0.      0.      0.     32.0853  0.9771 -1.2743 -1.2259]
 [ 0.      0.      0.      0.      0.      0.      0.     30.1255 -1.5697 -0.5702]
 [ 0.      0.      0.      0.      0.      0.      0.      0.     30.4768 -0.3046]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.     32.2902]]


Set up an ill-conditioned matrix.

In [13]:
epsilon = 1e-8
A = np.array([[1, 1, 1],
              [epsilon, 0, 0],
              [0, epsilon, 0],
              [0, 0, epsilon]])

Run the classical $QR$.

In [14]:
Q, R = classical_QR(A)

print(np.allclose(A,Q@R))
print(Q)
print(Q.T@Q)
print(R)

True
[[ 1.      0.      0.    ]
 [ 0.     -0.7071 -0.7071]
 [ 0.      0.7071  0.    ]
 [ 0.      0.      0.7071]]
[[ 1.  -0.  -0. ]
 [-0.   1.   0.5]
 [-0.   0.5  1. ]]
[[1. 1. 1.]
 [0. 0. 0.]
 [0. 0. 0.]]


Run the modified $QR$.

In [15]:
Q, R = modified_QR(A)

print(np.allclose(A,Q@R))
print(Q)
print(Q.T@Q)
print(R)

True
[[ 1.      0.      0.    ]
 [ 0.     -0.7071 -0.4082]
 [ 0.      0.7071 -0.4082]
 [ 0.      0.      0.8165]]
[[ 1. -0. -0.]
 [-0.  1.  0.]
 [-0.  0.  1.]]
[[1. 1. 1.]
 [0. 0. 0.]
 [0. 0. 0.]]
